In [5]:
import pandas as pd
import os
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt
import gensim
import numpy as np
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary

In [12]:
outputPath='data//'
docs=[]
txtNames=[]
for txtName in os.listdir(outputPath):
    txtNames.append(txtName)

for txtName in txtNames:
    txtFile=open(outputPath+txtName)
    txtFile.seek(0)
    docs.append(txtFile.read())
    txtFile.close()

In [52]:
black_list=['diputado','secretario','punto','día','dia','acta','sesion','honorable','señor','periodo','sesiones','presidente','presidenta','sesion','sesiones','correspondencia','orden','pleno','proyecto','ministro','diputados','miembros','miembro','proyectos','funcionario','funcionarios','anteproyecto','anteproyectos'] 

In [53]:
def cleaner(word):
  word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', word, flags=re.MULTILINE)#Direcciones Web. 
  word = re.sub(r'\#\.', '', word)#Numerales seguidos de puntos.
  word = re.sub(r'\n', '', word)#Saltos de linea
  word = re.sub(r',','', word)#Comas
  word = re.sub(r'\-', ' ', word)#Guiones.
  word = re.sub(r'\.{3}', ' ', word)#Puntos suspensivos.
  word = re.sub(r'á', 'a', word)#tildes.
  word = re.sub(r'é', 'e', word)
  word = re.sub(r'í', 'i', word)
  word = re.sub(r'ó', 'o', word)
  word = re.sub(r'ú', 'u', word) 
  word = re.sub(r'a{2,}', 'a', word)#Dos o más a seguidas -->a 
  word = re.sub(r'e{2,}', 'e', word)#Dos o más e seguidas -->a 
  word = re.sub(r'i{2,}', 'i', word)#Dos o más i seguidas -->a 
  word = re.sub(r'o{2,}', 'o', word)#Dos o más o seguidas -->a 
  word = re.sub(r'u{2,}', 'u', word)#Dos o más u seguidas -->a 
  word = re.sub(r'ja{2,}', 'ja', word)#Jajaja -->ja  
  word = re.sub('[^a-zA-Z]', ' ', word)#Todo lo que no sea palabras. 
  list_word_clean = []
  for w1 in word.split(" "):
    if  w1.lower() not in stopwords:
      list_word_clean.append(w1.lower())

  bigram_list = bigram[list_word_clean]
  out_text = lemmatization(" ".join(bigram_list))
  return out_text

In [54]:
def lemmatization(texts, allowed_postags=['NOUN']):
    texts_out = [ token.text for token in nlp(texts) if token.pos_ in 
                 allowed_postags and token.text not in black_list and len(token.text)>2]
    return texts_out

In [55]:
bigram = gensim.models.Phrases(docs)

In [56]:
import spacy.cli
spacy.cli.download("es_core_news_md")
import spacy
nlp = spacy.load('es_core_news_md')

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [57]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('spanish'))

additional_stopwords=set(black_list)

stopwords = stop.union(additional_stopwords)


In [58]:
docsCleaned=[]
for doc in docs:
    docsCleaned.append(cleaner(doc))

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [59]:
dictionary = Dictionary(docsCleaned)

In [60]:
dictionary.compactify()#Da nuevos ID. 
#Filtramos palabras que estén en minimo 2 documentos y máximo el 97% de documentos. 
dictionary.filter_extremes(no_below=2, no_above=0.97, keep_n=None)
dictionary.compactify()
#Creamos el Corpus
corpus = [dictionary.doc2bow(text) for text in docsCleaned]

In [61]:
#Función para mostrar los temas. 
def display_topics(model, model_type="lda"):
  for topic_idx, topic in enumerate(model.print_topics()):
    print ("Topic %d:" % (topic_idx))
    if model_type== "hdp":
      print (" ".join(re.findall( r'\*(.[^\*-S]+).?', topic[1])), "\n")
    else:
      print (" ".join(re.findall( r'\"(.[^"]+).?', topic[1])), "\n")

In [62]:
hdpmodel=HdpModel(corpus=corpus,id2word=dictionary,random_state=30)

In [63]:
display_topics(hdpmodel,model_type="hdp")

Topic 0:
transporte  momento  cosas  producto  carga  productos  programa  forma  arroz  tiendas 

Topic 1:
viviendas  vivienda  gente  cosas  plan  infraestructuras  nivel  casas  ciudad  vida 

Topic 2:
millones  presupuesto  recursos  ingresos  sector  impacto  forma  tiempo  agua  nivel 

Topic 3:
sector  comisionados  productor  copia  propuesta  compras  productores  octubre  banca  banco 

Topic 4:
debate  mujer  mujeres  agua  municipios  lectura  nivel  pueblos  medicina  veces 

Topic 5:
millones  modificaciones  honorables  gastos  ingresos  ingreso  gasto  vigencia  cuatrocientos  presupuesto 

Topic 6:
medicamentos  desabastecimiento  salud  ministra  registro  productos  problema  calidad  nivel  seguridad 

Topic 7:
carga  transporte  peso  dimensiones  vehiculo  vehiculos  permiso  enero  pesos  actividad 

Topic 8:
sector  trazabilidad  debate  uso  productos  relaciones  ores  productor  productores  registro 

Topic 9:
adultos  infancia  ejemplo  ministra  albergues 

In [64]:
hdpmodel.print_topics(num_topics=15, num_words=10)

[(0,
  '0.010*transporte + 0.009*momento + 0.009*cosas + 0.008*producto + 0.007*carga + 0.007*productos + 0.007*programa + 0.006*forma + 0.006*arroz + 0.006*tiendas'),
 (1,
  '0.010*viviendas + 0.009*vivienda + 0.009*gente + 0.007*cosas + 0.007*plan + 0.007*infraestructuras + 0.007*nivel + 0.007*casas + 0.007*ciudad + 0.006*vida'),
 (2,
  '0.019*millones + 0.012*presupuesto + 0.010*recursos + 0.008*ingresos + 0.006*sector + 0.006*impacto + 0.006*forma + 0.006*tiempo + 0.005*agua + 0.005*nivel'),
 (3,
  '0.013*sector + 0.012*comisionados + 0.011*productor + 0.008*copia + 0.008*propuesta + 0.007*compras + 0.006*productores + 0.006*octubre + 0.006*banca + 0.006*banco'),
 (4,
  '0.011*debate + 0.009*mujer + 0.009*mujeres + 0.008*agua + 0.008*municipios + 0.008*lectura + 0.007*nivel + 0.007*pueblos + 0.006*medicina + 0.005*veces'),
 (5,
  '0.088*millones + 0.047*modificaciones + 0.032*honorables + 0.020*gastos + 0.019*ingresos + 0.014*ingreso + 0.014*gasto + 0.008*vigencia + 0.008*cuatrocie

In [65]:
lsimodel = LsiModel(corpus=corpus, num_topics=15, id2word=dictionary)

In [66]:
display_topics(lsimodel)

Topic 0:
millones modificaciones honorables ingresos gastos ingreso gasto presupuesto vigencia propuesta 

Topic 1:
millones cosas transporte nivel carga sector momento gente tiempo debate 

Topic 2:
carga transporte dimensiones peso vehiculo permiso vehiculos enero pesos cargas 

Topic 3:
productor sector viviendas vivienda infraestructuras plan productores casas gente ciudad 

Topic 4:
medicamentos ministra desabastecimiento productor salud sector adultos viviendas comisionados gente 

Topic 5:
productos medicamentos recursos desabastecimiento adultos salud registro zona producto problema 

Topic 6:
acu modificaciones honorables millones presupuesto animales programa moneda valor zona 

Topic 7:
programa adultos arroz tiendas medicamentos desabastecimiento zona infancia junta animales 

Topic 8:
programa adultos derechos animales infancia albergues tiendas productos humanos ministra 

Topic 9:
acu debate moneda zona octubre valor junta productos copia centavos 

Topic 10:
acu animale

In [67]:
ldamodel= LdaModel(corpus=corpus,num_topics=15,id2word=dictionary)

In [68]:
display_topics(ldamodel)

Topic 0:
nivel lectura recursos tiempo comisionados sector desarrollo acuerdo instituciones gente 

Topic 1:
sector momento tiempo cosas nivel millones recursos honorables comisionados veces 

Topic 2:
debate sector millones cosas comisionados uso honorables gente momento caso 

Topic 3:
millones honorables debate cosas ejemplo recursos tiempo programa sector desarrollo 

Topic 4:
millones transporte carga sector forma gente acu cosas momento proceso 

Topic 5:
millones honorables nivel transporte modificaciones carga momento lectura recursos equipo 

Topic 6:
millones modificaciones honorables gastos ingresos lectura sector debate recursos cosas 

Topic 7:
millones modificaciones honorables gastos ingresos ingreso gasto desarrollo lectura caso 

Topic 8:
millones debate lectura tiempo sector momento nivel uso cosas proceso 

Topic 9:
cosas debate nivel transporte forma tiempo sector lectura productos uso 

Topic 10:
recursos desarrollo vida cosas momento instituciones nivel comisionad